# MIMIC Notes Pre-Processing

Pre-processing MIMIC notes for language model and CUIs

Below is a list of redacted items with an example and the replacement token.

Redacted items:
* [x] First Name: `[**First Name (Titles) 137**]`, `t_firstname`
* [x] Last Name: `[**Last Name (Titles) **]`, `t_lastname`
* [x] Initials: `[**Initials (NamePattern4) **]`, `t_initials`
* [x] Name: `[**Name (NI) **]`, `t_name`
* [x] Doctor First Name: `[**Doctor First Name 1266**]`, `t_doctor_firstname`
* [x] Doctor Last Name: `[**Doctor Last Name 1266**]`, `t_doctor_lastname`
* [x] Known Last Name: `[**Known lastname 658**]`, `t_lastname`
* [x] Hospital: `[**Hospital1 **]`, `t_hospital`
* [x] Hospital Unit Name: `**Hospital Unit Name 10**`, `t_hospital`
* [x] Company: `[**Company 12924**]`, `t_workplace`
* [x] University/College: `[**University/College **]`, `t_workplace`
* [x] Date of format YYYY-M-DD: `[**2112-4-18**]`, `t_fulldate`
* [x] Year: `[**Year (4 digits) **]`, `t_year`
* [x] Year YYYY format: `[**2119**]`, `t_year` - I use a regex `\b\d{4}\b` that will match **any** 4 digits which might be problematic, but for the most part 4 digits by itself seems to indicate a year.
* [x] Date of format M-DD: `[**6-12**]`, `[**12/2151**]`, `t_monthday`
* [x] Month/Day: `[**Month/Day (2) 509**]`, `t_monthday`
* [x] Month (only): `[**Month (only) 51**]`, `t_month`
* [x] Holiday: `[**Holiday 3470**]`, `t_month`
* [x] Date Range: `[**Date range (1) 7610**]`, `t_daterange`
* [x] Country: `[**Country 9958**]`, `t_country`
* [x] State: `[**State 3283**]`, `t_state`
* [x] Location: `**Location (un) 2432**`, `t_location`
* [x] Telephone/Fax: `[**Telephone/Fax (3) 8049**]`, `t_phone`
* [x] Clip Number: `[**Clip Number (Radiology) 29923**]`, `t_radclip_id`
* [x] Pager Numeric Identifier: `[**Numeric Identifier 6403**]`, `t_pager_id`
* [x] Pager Number: `[**Pager number 13866**]`, `t_pager_id`
* [x] Social Security Number: `[**Security Number 10198**]`, `t_ssn`
* [x] Serial Number: `[**Serial Number 3567**]`, `t_sn`
* [x] Medical Record Number: `[**Medical Record Number **]`, `t_mrn`
* [x] Provider Number: `[**Provider Number 12521**]`, `t_provider_no`
* [x] Age over 90: `[**Age over 90 **]`, `t_oldage`
* [x] Time: `12:52 PM`, split into 6 segments by the hour and replace with the following tokens: `midnight, dawn, forenoon, afternoon, dusk, night`
* Just numbers: `[** 7901**]`
* Wardname
* Pharmacy MD Number* 

## Imports and Inits

In [1]:
import pandas as pd
import psycopg2
import numpy as np
import re
import random
import datetime

In [2]:
from process_notes import *

## Grab sample data from MIMIC

In [3]:
cats = pd.read_csv('cats.csv')
max_limit = 100000

queries = []
for category, n_notes in zip(cats['category'], cats['number_of_notes']):
    limit = min(max_limit, n_notes) if max_limit > 0 else n_notes
    if limit == max_limit:
        q = f"""
        select category, text from correctnotes where category=\'{category}\' order by random() limit {limit};
        """
    else:
        q = f"""
        select category, text from correctnotes where category=\'{category}\';
        """
    queries.append(q)

In [4]:
%%time
dfs = []

con = psycopg2.connect(dbname='mimic', user='sudarshan', host='/var/run/postgresql')
for q in queries:
    df = pd.read_sql_query(q, con)
    dfs.append(df)
con.close()
    
df = pd.concat(dfs)
df.reset_index(inplace=True, drop=True)
# df.set_index('row_id', inplace=True)
print(df.shape)

(604352, 2)
CPU times: user 1.48 s, sys: 1.53 s, total: 3.01 s
Wall time: 5min 28s


In [5]:
pat = re.compile(r'\[\*\*(.*?)\*\*\]', re.IGNORECASE)
tpat = re.compile(r'\[\*\*(\d{2})\*\*\] \b[a|p].?m.?\b', re.IGNORECASE)
ypat = re.compile(r'-?\byears? ?-?old\b|\by(?:o|r)*[ ./-]*o(?:ld)?\b', re.IGNORECASE)

In [6]:
%%time
df['scrubbed'] = df['text'].apply(process_note)

CPU times: user 7min 2s, sys: 189 ms, total: 7min 2s
Wall time: 7min 2s


In [ ]:
test = df.iloc[33296]['text']
# for m in tpat.finditer(test):
#     print(m)

print(test)

In [ ]:
out = process_note(test)
for m in ypat.finditer(out):
    print(m)

print(out)

In [ ]:
for i, row in df.iterrows():
    if len(tpat.findall(row['text'])) != 0:
        print(i, tpat.findall(row['text']))

In [ ]:
for i, row in df.iterrows():
    if len(tpat.findall(row['scrubbed'])) != 0:
        print(i, tpat.findall(row['scrubbed']))

In [ ]:
for i, row in df.iterrows():
    if len(pat.findall(row['scrubbed'])) != 0:
        print(i, pat.findall(row['scrubbed']))

497 [' ']
500 [' ']
527 [' ']
707 [' ']
760 [' ']
48145 [' ']
104297 [' ']
105118 [' ']
108796 [' ']
113018 [' ']
113095 [' ']
113096 [' ']
113098 [' ']
113611 [' ']
123047 [' ', ' ']
133752 [' ']
134763 [' ']
135094 [' ']
135943 [' ']
136372 [' ']
136405 [' ']
137532 [' ']
139311 [' ']
142900 [' ', ' ']
147323 [' ', ' ']
148035 [' ']
148499 [' ']
149890 [' ']
150858 [' ']
151832 [' ']
152102 [' ']
152947 [' ']
155234 [' ']
156665 [' ']
156721 [' ']
156790 [' ', ' ']
157004 [' ', ' ', ' ', ' ', ' ']
157301 [' ', ' ']
157499 [' ']
157575 [' ']
157680 [' ']
157797 [' ']
157811 [' ', ' ']
158816 [' ']
159174 [' ']
159702 [' ']
159795 ['']
159909 [' ']
161381 [' ', ' ']
161476 [' ']
161617 [' ']
162954 ['']
163373 [' ', ' ']
163398 [' ']
163674 [' ']
164327 [' ']
165324 [' ']
165420 [' ']
166050 [' ']
166123 [' ']
166418 [' ']
166990 [' ']
167125 [' ', ' ']
169217 [' ']
169942 ['']
170518 [' ', ' ']
171196 ['']
171422 [' ']
172735 [' ']
173249 [' ']
173596 [' ']
173763 [' ']
174117 ['']
17